# 安裝

In [ ]:
!pip install gurobipy  # install gurobipy, if not already installed
import gurobipy as gp  # import the installed package

In [ ]:
!pip install gradio gurobipy openpyxl
!pip install pyngrok


# 前端

In [ ]:
import gradio as gr
import requests
import pandas as pd

API_URL = "http://127.0.0.1:5000/solve"

def solve_and_display(model_file, time_limit):
    try:
        # 上傳文件到後端
        with open(model_file.name, 'rb') as f:
            response = requests.post(
                API_URL,
                files={"file": f},
                data={"time_limit": time_limit}
            )

        # 處理後端返回的結果
        result = response.json()

        # 檢查錯誤
        if 'error' in result:
            error_message = f"錯誤: {result['error']}"
            if "infeasible_constraints" in result:
                error_message += "\n不可行約束:\n" + "\n".join(result["infeasible_constraints"])
            logs = result.get("log", "無日誌輸出")
            return error_message, None, logs

        # 處理成功結果
        variables = pd.DataFrame(result['variables'])
        variables.rename(columns={"VarName": "Variable", "Value": "Value"}, inplace=True)
        objective = result["objective"]
        status = result["status"]
        logs = result.get("log", "無日誌輸出")

        # 組合求解摘要
        description = f"目標值 (Objective): {objective}\n狀態碼 (Status): {status}"

        return description, variables, logs

    except requests.exceptions.RequestException as e:
        return f"請求失敗: {str(e)}", None, None
    except Exception as e:
        return f"未知錯誤: {str(e)}", None, None


# 定義 Gradio 介面
with gr.Blocks() as demo:
    gr.Markdown("## Gurobi Model Runner")
    with gr.Row():
        model_file = gr.File(label="上傳 Gurobi 模型文件")
        time_limit = gr.Number(label="時間限制 (秒, 0 表示無限制)", value=0)
    solve_button = gr.Button("求解模型")
    with gr.Row():
        result_text = gr.Textbox(label="求解摘要", lines=5, interactive=False)
    with gr.Row():
        variable_table = gr.Dataframe(label="變數結果", headers=["Variable", "Value"], interactive=False)
    with gr.Row():
        log_text = gr.Textbox(label="日誌輸出", lines=15, interactive=False)

    solve_button.click(
        solve_and_display,
        inputs=[model_file, time_limit],
        outputs=[result_text, variable_table, log_text]
    )

demo.launch()


# 後端

In [ ]:
from flask import Flask, request, jsonify
import gurobipy as gp
from gurobipy import GRB
import tempfile
import os

app = Flask(__name__)

@app.route('/solve', methods=['POST'])
def solve_model():
    try:
        # 接收請求
        data = request.files['file']
        time_limit = float(request.form.get('time_limit', 0))

        # 保存模型文件
        temp_dir = tempfile.mkdtemp()
        model_file_path = os.path.join(temp_dir, data.filename)
        with open(model_file_path, 'wb') as f:
            f.write(data.read())

        # 讀取並求解模型
        model = gp.read(model_file_path)
        if time_limit > 0:
            model.setParam(GRB.Param.TimeLimit, time_limit)

        # 設置日誌文件
        log_output = []
        with tempfile.NamedTemporaryFile(delete=False) as log_file:
            model.setParam("LogFile", log_file.name)
            model.optimize()
            with open(log_file.name, "r") as log:
                log_output = log.readlines()

        # 檢查求解狀態
        if model.Status == GRB.INFEASIBLE:
            # 執行不可行性分析 (IIS)
            model.computeIIS()
            infeasible_constraints = [
                c.ConstrName for c in model.getConstrs() if c.IISConstr
            ]
            return jsonify({
                "error": "模型不可行",
                "infeasible_constraints": infeasible_constraints,
                "log": "".join(log_output)
            }), 400

        # 收集結果
        variables = [{"VarName": v.VarName, "Value": v.X} for v in model.getVars()]
        objective = model.ObjVal if model.Status == GRB.OPTIMAL else None
        status = model.Status

        # 返回結果
        return jsonify({
            "variables": variables,
            "objective": objective,
            "status": status,
            "log": "".join(log_output)
        })

    except gp.GurobiError as e:
        return jsonify({"error": f"Gurobi Error: {str(e)}"}), 500
    except Exception as e:
        return jsonify({"error": f"未知錯誤: {str(e)}"}), 500


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
